In [1]:
import torch

In [2]:
from comet_ml import Experiment, Optimizer

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import trange
import torch
import seaborn as sns
import os
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import sklearn
import time
import datetime

plt.style.use("seaborn")
torch.set_default_dtype(torch.float32)

In [4]:
import node

In [5]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [6]:
from tabular_hypernet import HypernetworkPCA, TrainingModes, Hypernetwork
from tabular_hypernet.modules import SimpleNetwork
from tabular_hypernet.training_utils import train_slow_step, train_model, train_carthesian
from tabular_hypernet.interfaces import HypernetworkSklearnInterface, SimpleSklearnInterface
# from ipynb.fs.defs.MNIST_benchmark import test_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [7]:
from sklearn.metrics import classification_report, balanced_accuracy_score

In [8]:
import imblearn

In [9]:
import pyhopper

In [10]:
from loguru import logger

logger.add("log_Lymf.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

1

In [11]:
import seaborn as sns

In [12]:
DEVICE="cuda:0"

In [13]:
from loguru import logger

logger.add("log.txt", format='{time:YYYY-MM-DD HH:mm:ss.SSS} | {message}')

2

In [14]:
lymphography = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data', header=None)
lymphography



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
1,2,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
2,3,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
4,2,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,3,3,2,1,1,2,2,1,2,1,2,2,4,3,5,2,2,2,4
144,2,2,1,1,1,1,1,1,1,1,1,1,1,1,3,1,2,2,1
145,3,2,2,1,1,1,2,1,2,1,3,3,3,3,8,3,2,2,4
146,2,2,1,1,1,1,1,1,2,1,2,2,4,2,2,1,2,2,1


In [15]:
lymphography[14].value_counts()



8    45
4    31
5    28
3    19
2    14
7     7
6     2
1     2
Name: 14, dtype: int64

# Load data

In [16]:
DATA = "Lymphography"

In [18]:
lol = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data', header=None)[0]

In [19]:
unique, counts = np.unique(lol.values, return_counts=True)

print(dict(zip(unique, counts)))

{1: 2, 2: 81, 3: 61, 4: 4}


In [15]:
if DATA == "Ionosphere":
    ionosphere = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data', header=None)
    ionosphere = ionosphere.drop(1, axis=1)
    X = ionosphere.values[:, :-1].astype(float)
    y = ionosphere.values[:, -1]
    y = LabelEncoder().fit_transform(y).astype(int)
    #(351, 33) 2 #auc roc
    GS_METRIC = "roc_auc"
    
elif DATA == "Libras":
    libras = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.data', header=None)
    X = libras.values[:, :-1].astype(float)
    y = libras.values[:, -1].astype(int)
    y -= 1
    #(360, 90) 15
    GS_METRIC = "balanced_accuracy"
    
elif DATA == "Lymphography":
    lymphography = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data', header=None)
    
    lymphography = lymphography[lymphography[0] != 4]
    lymphography = lymphography[lymphography[0] != 1]
    lymphography[0] = lymphography[0] - 1
    
    X = lymphography.values[:, 1:].astype(float)
    y = lymphography.values[:, 0].astype(int)
    y -= 1
    
    #(148, 18) 4
    GS_METRIC = "balanced_accuracy"
    
#     X, y = imblearn.over_sampling.SMOTE(
#                         sampling_strategy={0:10, 3:20}, 
#                         random_state=42,
#                         k_neighbors=1,
#                     ).fit_resample(X, y)
    
    
print(X.shape, len(np.unique(y)))


(142, 18) 2


In [16]:
n_classes = len(np.unique(y))
n_classes

2

In [17]:
unique, counts = np.unique(y, return_counts=True)

print(dict(zip(unique, counts)))

{0: 81, 1: 61}


In [18]:
n_features = X.shape[1]
n_features

18

In [19]:
max_size = int(len(X)*0.7)

In [20]:
print('dataset size:', len(X), '|', 'max training size:', max_size)

dataset size: 142 | max training size: 99


In [21]:
TEST_RUN = False

# Preprocess

In [23]:
np.random.seed(42)

def prepare_data(X, y, size=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, stratify=y)
    # X_train, y_train = imblearn.over_sampling.RandomOverSampler(random_state=42).fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train, X_test, y_train, y_test = [torch.from_numpy(x) for x in [X_train, X_test, y_train, y_test]]
    
    return X_train, X_test, y_train, y_test

In [24]:
X_train, X_test, y_train, y_test = prepare_data(X, y, size=max_size)

## Benchmark

In [25]:
criterion = torch.nn.CrossEntropyLoss()

In [26]:
def _summarize_results(y_pred, y_score, y_test, labels):
    results = []
    for idx, label in enumerate(labels):
        y_pred_filt = y_pred[y_test==idx]
        y_test_filt = y_test[y_test==idx]
        acc = (y_pred_filt==y_test_filt.numpy()).sum()/len(y_test_filt)*100
        results.append({
            "Class": label,
            "Metric": acc
        })
        
    acc = (y_pred==y_test.numpy()).sum()/len(y_test)*100    
    results.append({
        "Class": "Total",
        "Metric": acc
    })
    
    
    results.append({
        "Class": "balanced_accuracy",
        "Metric": balanced_accuracy_score(y_test, torch.from_numpy(y_pred)).item()*100
    })
    
    try:
        results.append({
            "Class": "F1 score",
            "Metric": f1_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "roc_auc",
            "Metric": roc_auc_score(y_test, torch.from_numpy(y_score[:, 1])).item()*100
        })
        results.append({
            "Class": "Precision",
            "Metric": precision_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
        results.append({
            "Class": "Recall",
            "Metric": recall_score(y_test, torch.from_numpy(y_pred)).item()*100
        })
    except ValueError:
        pass
    return results

def test_model(model_fn, data, train_size, label_encoder=None, iters=10, as_numpy=False):
    if TEST_RUN:
        iters = 1
        
    if label_encoder is not None:
        labels = label_encoder.classes_
    else:
        labels = sorted(pd.unique(data[1]))
    
    results = []

    for i in range(iters):
        print('iter', i+1, 'of', iters)
        X_train, X_test, y_train, y_test = prepare_data(*data, train_size)
        
        model = model_fn()

        if as_numpy:
            model.fit(X_train.numpy(), y_train.numpy());
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_score = model.predict_proba(X_test)
        results.extend(_summarize_results(y_pred, y_score, y_test, labels))
        
        del model
        torch.cuda.empty_cache()

    dframe = pd.DataFrame.from_dict(results)
#     sns.violinplot(data=dframe[dframe["Class"]!="Loss"], y="Class", x="Metric", orient='h')
    return dframe

### Param search

In [27]:
# def grid_search_best_params(model_fn, param_grid, data_size=max_size, scoring=GS_METRIC):
#     if TEST_RUN:
#         tmp = {k: [v[0]] for k, v in param_grid.items()}
#         param_grid = tmp
        
#     X_train, X_test, y_train, y_test = prepare_data(X, y, data_size)

#     cv_clf = GridSearchCV(
#         estimator=model_fn(), 
#         param_grid=param_grid,
#         scoring=scoring, 
#         return_train_score=True,
#         verbose=1, 
#         cv=5,
#     )

#     res = cv_clf.fit(X_train, y_train)

#     print(f'best params for {DATA}', res.best_params_)
#     print('mean_train_score', cv_clf.cv_results_['mean_train_score'].mean())
#     print('std_train_score', cv_clf.cv_results_['std_train_score'].mean())

#     predictions = cv_clf.predict(X_test) 
#     print(classification_report(y_test, predictions))
    
#     with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
#             f.write(str(res.best_params_) + ", " + str(balanced_accuracy_score(y_test, predictions)) + "\n")
    
#     print(f"{DATA}_{model_fn.__name__}_{res.best_params_}")
    
#     return res.best_params_

In [28]:
def pyhopper_best_params(model_fn, param_grid, data_size=max_size, metric=GS_METRIC, time="30m", default_params={}):
    if TEST_RUN:
        time = 60
        
    logger.info(f"pyhopper_best_params {model_fn.__name__}")
    
    def objective(params):
        torch.cuda.empty_cache()
    
        model_results = test_model(
                        model_fn(
                            **default_params,
                            **params
                        ),
                        (X, y),
                        data_size,
                        None, 5)
        with open(f"{DATA}_{model_fn.__name__}_params.txt", "a") as f:
            f.write(str(params) + ", " + str(model_results[model_results["Class"]==metric]["Metric"].mean()) + "\n")
        return model_results[model_results["Class"]==metric]["Metric"].mean()

    from pyhopper.callbacks import History
    search = pyhopper.Search(param_grid)

    best_params = search.run(objective, "maximize", time, n_jobs="1x per-gpu", seeding_ratio=0.5)
    
    with open(f"{DATA}_{model_fn.__name__}_best_params.txt", "a") as f:
            f.write(str(best_params))
    
    print(f"{DATA}_{model_fn.__name__}_{best_params}")
    return best_params


# TRAIN MODELS

### XGBoost

In [28]:
from xgboost import XGBClassifier

In [29]:
import xgboost
xgboost.__version__

'1.6.0'

In [30]:
def get_xgboost(**params):
    random_seed = np.random.randint(1024)
    def _inner(**args):
        return XGBClassifier(
            verbosity=0,
            random_state=random_seed,
            use_label_encoder=False,
            **params,
            **args
        )
    return _inner    

#### Hyperparam tuning

In [31]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_depth': pyhopper.choice([2, 3, 5, 10, 15]),
                'learning_rate': pyhopper.float(1e-5,1e-1, log=True),
                'min_child_weight': pyhopper.choice([1, 2, 4, 8, 16, 32]),
                'gamma': pyhopper.choice([0, 0.001, 0.1, 1]),
             }

xgbt_best1 = pyhopper_best_params(get_xgboost, param_grid, time="90m")

2022-11-16 01:49:55.568 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5
iter 4 of 5


...

iter 4 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 5 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 86.47  : 1     : 2.95 s          
Random seeding    : 91     : 1k    : 44:54 (m:s)     
Local sampling    : 91.8   : 1k    : 44:54 (m:s)     
Duplicates        : -      : 436   : -               
----------------  : ----   : ----  : ----            
Total             : 91.8   : 2k    : 01:29:57 (h:m:s)
Lymphography_get_xgboost_{'n_estimators': 1700, 'max_depth': 10, 'learning_rate': 0.0167799646

In [32]:
param_grid = {
                'subsample': pyhopper.choice([0.5, 0.6, 0.7, 0.8, 0.9, 1]),
                'reg_lambda': pyhopper.float(1e-5, 10, init=0, log=True),
                'reg_alpha': pyhopper.float(1e-5, 10, init=0, log=True),
             }


xgbt_best2 = pyhopper_best_params(get_xgboost, param_grid, time="90m", default_params=xgbt_best1)

2022-11-16 03:19:52.925 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params get_xgboost


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter

iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter

...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Best Params

'Ionosphere' {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 1500}


In [33]:
xgboost_best = {**xgbt_best1, **xgbt_best2}

In [34]:
data_size = max_size

xgb_dframe = test_model(get_xgboost(**xgboost_best),
                        (X, y),
                        data_size,
                        label_encoder=None, iters=20)

iter 1 of 20
iter 2 of 20
iter 3 of 20
iter 4 of 20
iter 5 of 20
iter 6 of 20
iter 7 of 20
iter 8 of 20
iter 9 of 20
iter 10 of 20
iter 11 of 20
iter 12 of 20
iter 13 of 20
iter 14 of 20
iter 15 of 20
iter 16 of 20
iter 17 of 20
iter 18 of 20
iter 19 of 20
iter 20 of 20


In [35]:
res = xgb_dframe[xgb_dframe["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

99: 86.86 ~ 2.42 (max: 90.70)


In [36]:
xgb_dframe.groupby(['Class']).mean()

,Metric
Class,
0,91.600000
1,80.277778
F1 score,83.409646
Precision,88.282499
Recall,80.277778
Total,86.860465
balanced_accuracy,85.938889
roc_auc,92.933333


### NODE

In [37]:
from qhoptim.pyt import QHAdam

In [38]:
def node_fn(layer_dim=128, num_layers=1, depth=3):
    def _inner():
        network = torch.nn.Sequential(
            node.DenseBlock(X_train.shape[1], 
                            layer_dim=layer_dim,
                            num_layers=num_layers, 
                            tree_dim=n_classes+1, 
                            depth=depth, 
                            flatten_output=False,
                            choice_function=node.entmax15, 
                            bin_function=node.entmoid15
                           ),
            node.Lambda(lambda x: x.mean(dim=1))
        )
        
        
        network = network.to(DEVICE)
        network.device=DEVICE
        
        with torch.no_grad():
            res = network(torch.as_tensor(X_train, device=DEVICE).to(torch.float32))
        
            
        optimizer_params = { 'nus':(0.7, 1.0), 'betas':(0.95, 0.998) }
        optim = QHAdam(network.parameters(), **optimizer_params)
            
        network = SimpleSklearnInterface(network, device=DEVICE, epochs=150, batch_size=32)
        network.optimizer = optim
        return network
    return _inner

#### Tune hyperparams

In [39]:
# 'layer_dim': hp.quniform('layer_dim', 100, 1200, 100),
# 'num_layers': hp.quniform('num_layers', 1, 4, 1),
# 'depth': hp.quniform('depth', 2, 7, 1)
                    
param_grid = {
    'layer_dim': pyhopper.int(64, 1024, power_of=2),
    'num_layers': pyhopper.int(1, 5),
    'depth': pyhopper.int(2, 7),
}

node_best = pyhopper_best_params(node_fn, param_grid, time="90m")
node_best

2022-11-16 04:50:03.988 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params node_fn


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."
/home/z1157095/anaconda3/en

iter 2 of 5


...

/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."


iter 5 of 5


/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:72: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
/home/z1157095/hypernet-cnn/deps/node/node/lib/odst.py:113: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."


====================== Summary ======================
Mode              : Best f : Steps : Time            
----------------  : ----   : ----  : ----            
Initial solution  : 87.58  : 1     : 02:42 (m:s)     
Random seeding    : 87.67  : 9     : 43:32 (m:s)     
Local sampling    : 88.53  : 17    : 41:46 (m:s)     
Duplicates        : -      : 12    : -               
----------------  : ----   : ----  : ----            
Total             : 88.53  : 39    : 01:28:01 (h:m:s)
Lymphography_node_fn_{'layer_dim': 128, 'num_layers': 2, 'depth': 4}


{'layer_dim': 128, 'num_layers': 2, 'depth': 4}

#### Use best hyperparams

In [40]:
%%capture --no-stdout

size=max_size

node_results = test_model(node_fn(**node_best),
                    (X, y),
                    size,
                    label_encoder=None, iters=20)
res = node_results[node_results["Class"]=="roc_auc"].reset_index(drop=True)["Metric"]
print(f"{size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

iter 1 of 20
iter 2 of 20
iter 3 of 20
iter 4 of 20
iter 5 of 20
iter 6 of 20
iter 7 of 20
iter 8 of 20
iter 9 of 20
iter 10 of 20
iter 11 of 20
iter 12 of 20
iter 13 of 20
iter 14 of 20
iter 15 of 20
iter 16 of 20
iter 17 of 20
iter 18 of 20
iter 19 of 20
iter 20 of 20
99: 89.72 ~ 4.97, (max: 96.89)


### Dropout Neural network

In [41]:
def network_fn1(epochs=100, drop1=0.3, drop2=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=3e-4)
        return network
    return _inner



def network_fn2(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=3e-4)
        return network
    return _inner



def network_fn3(epochs=100, drop1=0.3, drop2=0.5, drop3=0.5, drop4=0.5, batch_size=32, lr=3e-4):
    
    def _inner():
        network = torch.nn.Sequential(
                        torch.nn.Dropout(drop1),
                        torch.nn.Linear(n_features, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop2),
                        torch.nn.Linear(64, 128),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop3),
                        torch.nn.Linear(128, 64),
                        torch.nn.ReLU(),
                        torch.nn.Dropout(drop4),
                        torch.nn.Linear(64, n_classes)
                    ).to(DEVICE).train()

        network = SimpleSklearnInterface(network, epochs=epochs, batch_size=batch_size, lr=3e-4)
        return network
    return _inner

#### Find Hyperparams

### Dropout 1 layer

In [42]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn1_best_params = pyhopper_best_params(network_fn1, param_grid, time="60m")
nn_fn1_best_params

2022-11-16 06:27:36.414 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params network_fn1


  0%|          | [00:00<?]

Search is scheduled for 01:00:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 150, 'drop1': 0.5, 'drop2': 0.1, 'lr': 0.003, 'batch_size': 64}

In [43]:
data_size = max_size

nn1_results = test_model(network_fn1(**nn_fn1_best_params),
                (X, y),
                data_size,
                None, 20)

iter 1 of 20
iter 2 of 20
iter 3 of 20
iter 4 of 20
iter 5 of 20
iter 6 of 20
iter 7 of 20
iter 8 of 20
iter 9 of 20
iter 10 of 20
iter 11 of 20
iter 12 of 20
iter 13 of 20
iter 14 of 20
iter 15 of 20
iter 16 of 20
iter 17 of 20
iter 18 of 20
iter 19 of 20
iter 20 of 20


In [44]:
res = nn1_results[nn1_results["Class"]=="Balanced Acc score"]["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

99: nan ~ nan (max: nan)


### Dropout 2 layers

In [45]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn2_best_params = pyhopper_best_params(network_fn2, param_grid, time="70m")
nn_fn2_best_params

2022-11-16 07:35:32.319 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params network_fn2


  0%|          | [00:00<?]

Search is scheduled for 01:10:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5

{'epochs': 100,
 'drop1': 0.1,
 'drop2': 0.1,
 'drop3': 0.5,
 'lr': 0.0003,
 'batch_size': 64}

In [46]:
data_size = max_size

nn2_results = test_model(network_fn2(**nn_fn2_best_params),
                (X, y),
                data_size,
                None, 20)

iter 1 of 20
iter 2 of 20
iter 3 of 20
iter 4 of 20
iter 5 of 20
iter 6 of 20
iter 7 of 20
iter 8 of 20
iter 9 of 20
iter 10 of 20
iter 11 of 20
iter 12 of 20
iter 13 of 20
iter 14 of 20
iter 15 of 20
iter 16 of 20
iter 17 of 20
iter 18 of 20
iter 19 of 20
iter 20 of 20


### Dropout 3 layers

In [47]:
param_grid = {
                "epochs": pyhopper.choice([100, 150]),
                "drop1": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop2": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop3": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "drop4": pyhopper.choice([0.1, 0.3, 0.5, 0.7], is_ordinal=True),
                "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
                "batch_size": pyhopper.choice([32, 64]),
             }

nn_fn3_best_params = pyhopper_best_params(network_fn3, param_grid, time="75m")
nn_fn3_best_params

2022-11-16 08:49:43.487 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params network_fn3


  0%|          | [00:00<?]

Search is scheduled for 01:15:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter

{'epochs': 150,
 'drop1': 0.1,
 'drop2': 0.7,
 'drop3': 0.1,
 'drop4': 0.7,
 'lr': 0.0003,
 'batch_size': 64}

In [48]:
data_size = max_size

nn3_results = test_model(network_fn3(**nn_fn3_best_params),
                (X, y),
                data_size,
                None, 20)

iter 1 of 20
iter 2 of 20
iter 3 of 20
iter 4 of 20
iter 5 of 20
iter 6 of 20
iter 7 of 20
iter 8 of 20
iter 9 of 20
iter 10 of 20
iter 11 of 20
iter 12 of 20
iter 13 of 20
iter 14 of 20
iter 15 of 20
iter 16 of 20
iter 17 of 20
iter 18 of 20
iter 19 of 20
iter 20 of 20


### HypernetworkPCA

In [49]:
def network_pca_fn(epochs=100, masks_no=100, mask_size=100, target_size=100, n_comp=5, lr=3e-4, batch_size=64):
    def _inner():
        hypernet = HypernetworkPCA(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)], 
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_comp, 32), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(32, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                        input_size=n_features
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=False, lr=3e-4)
        return network
    return _inner

In [50]:
def figure_to_html():
    tmpfile = BytesIO()
    plt.gcf().savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

    html = '<img src=\'data:image/png;base64,{}\'>'.format(encoded)
    return html

In [51]:
X.shape[1] // 7

2

#### Find hyperparams

In [52]:
param_grid = {
    "epochs": pyhopper.choice([100, 150]),
    "masks_no": pyhopper.int(20, 100, multiple_of=10),
    "mask_size": pyhopper.int(3, 18),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "n_comp": pyhopper.int(3, 15),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([32, 64]),

}

hp_pca_best_params = pyhopper_best_params(network_pca_fn, param_grid, time="90m")
hp_pca_best_params

2022-11-16 10:11:44.861 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params network_pca_fn


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 128])
18
iter 4 of 5
torch.Size([1, 128])
18
iter 5 of 5
torch.Size([1, 128])
18
iter 1 of 5
torch.Size([1, 128])
18


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:527: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


iter 2 of 5
torch.Size([1, 128])
18


...

/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:527: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


iter 5 of 5
torch.Size([1, 128])
18


/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:527: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 128])
18
iter 4 of 5
torch.Size([1, 128])
18
iter 5 of 5
torch.Size([1, 128])
18
iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 128])
18
iter 4 of 5
torch.Size([1, 128])
18
iter 5 of 5
torch.Size([1, 128])
18
iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 128])
18
iter 4 of 5
torch.Size([1, 128])
18
iter 5 of 5
torch.Size([1, 128])
18
iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 128])
18
iter 4 of 5
torch.Size([1, 128])
18
iter 5 of 5
torch.Size([1, 128])
18
iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 128])
18
iter 4 of 5
torch.Size([1, 128])
18
iter 5 of 5
torch.Size([1, 128])
18
iter 1 of 5
torch.Size([1, 128])
18
iter 2 of 5
torch.Size([1, 128])
18
iter 3 of 5
torch.Size([1, 1

{'epochs': 150,
 'masks_no': 50,
 'mask_size': 5,
 'target_size': 50,
 'n_comp': 11,
 'lr': 0.0003,
 'batch_size': 32}

In [53]:
hp_pca_best_params

{'epochs': 150,
 'masks_no': 50,
 'mask_size': 5,
 'target_size': 50,
 'n_comp': 11,
 'lr': 0.0003,
 'batch_size': 32}

'Libras'
{'epochs': 150,
 'masks_no': 70,
 'mask_size': 20,
 'target_size': 10,
 'n_comp': 10}

'Lymphography' {'epochs': 120, 'masks_no': 50, 'mask_size': 4, 'target_size': 20, 'n_comp': 6}

In [54]:
DATA

'Lymphography'

Ionosphere
{'epochs': 100, 'masks_no': 60, 'mask_size': 5, 'target_size': 10}


In [55]:
epochs = hp_pca_best_params['epochs']
masks_no = hp_pca_best_params['masks_no']
mask_size = hp_pca_best_params['mask_size']
target_size = hp_pca_best_params['target_size']
n_comp = hp_pca_best_params['n_comp']
data_size = max_size

nn_pca_results = test_model(network_pca_fn(target_size=target_size, mask_size=mask_size, masks_no=masks_no, n_comp=n_comp),
                (X, y),
                data_size,
                None, 20)

# exp.log_table("metrics.csv", nn_pca_results.groupby("Class").mean())
# exp.log_metric("f1_score", nn_pca_results.groupby("Class").mean().loc["F1 score"])

iter 1 of 20
torch.Size([1, 128])
18
iter 2 of 20
torch.Size([1, 128])
18
iter 3 of 20
torch.Size([1, 128])
18
iter 4 of 20
torch.Size([1, 128])
18
iter 5 of 20
torch.Size([1, 128])
18
iter 6 of 20
torch.Size([1, 128])
18
iter 7 of 20
torch.Size([1, 128])
18
iter 8 of 20
torch.Size([1, 128])
18
iter 9 of 20
torch.Size([1, 128])
18
iter 10 of 20
torch.Size([1, 128])
18
iter 11 of 20
torch.Size([1, 128])
18
iter 12 of 20
torch.Size([1, 128])
18
iter 13 of 20
torch.Size([1, 128])
18
iter 14 of 20
torch.Size([1, 128])
18
iter 15 of 20
torch.Size([1, 128])
18
iter 16 of 20
torch.Size([1, 128])
18
iter 17 of 20
torch.Size([1, 128])
18
iter 18 of 20
torch.Size([1, 128])
18
iter 19 of 20
torch.Size([1, 128])
18
iter 20 of 20
torch.Size([1, 128])
18


In [56]:
res = nn_pca_results[nn_pca_results["Class"]=="Total"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f} (max: {res.max():.2f})")

99: 84.42 ~ 5.70 (max: 100.00)


### Hypernetwork

In [30]:
def network_hp_fn(epochs=150, masks_no=100, mask_size=100, target_size=100, lr=3e-4, batch_size=64):
    def _inner():
        hypernet = Hypernetwork(
                        target_architecture=[(mask_size, target_size), (target_size, n_classes)],
                        test_nodes=masks_no,
                        architecture=torch.nn.Sequential(torch.nn.Linear(n_features, 64), 
                            torch.nn.ReLU(),
                            torch.nn.Linear(64, 128),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(),
                            torch.nn.Linear(128, 128),
                            torch.nn.ReLU(),
                        ),
                        mode=TrainingModes.CARTHESIAN,
                    ).to(DEVICE)    
        hypernet = hypernet.train()

        network = HypernetworkSklearnInterface(hypernet, device=DEVICE, epochs=epochs, batch_size=batch_size, verbose=False, lr=3e-4)
        return network
    return _inner

#### Find hyperparams

In [93]:
param_grid = {
    "epochs": pyhopper.choice([100, 150]),
    "masks_no": pyhopper.int(10, 400, multiple_of=10),
    "mask_size": pyhopper.int(2, 18),
    "target_size": pyhopper.choice([5, 10, 20, 50]),
    "lr": pyhopper.choice([3e-5, 3e-4, 3e-3, 3e-2, 3e-1]),
    "batch_size": pyhopper.choice([16, 32, 64]),
}

hp_best_params = pyhopper_best_params(network_hp_fn, param_grid, time="240m")
hp_best_params

2022-11-17 10:26:02.240 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params network_hp_fn


  0%|          | [00:00<?]

Search is scheduled for 04:00:00 (h:m:s)
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
iter 5 of 5
torch.Size([1, 128])
iter 1 of 5
torch.Size([1, 128])
iter 2 of 5
torch.Size([1, 128])
iter 3 of 5
torch.Size([1, 128])
iter 4 of 5
torch.Size([1, 128])
it

{'epochs': 100,
 'masks_no': 110,
 'mask_size': 12,
 'target_size': 20,
 'lr': 0.03,
 'batch_size': 16}

In [94]:
hp_best_params

{'epochs': 100,
 'masks_no': 110,
 'mask_size': 12,
 'target_size': 20,
 'lr': 0.03,
 'batch_size': 16}

#### Train using the best hyperparams

In [37]:
epochs = 100 #hp_best_params['epochs']
masks_no = 150 #hp_best_params['masks_no']
mask_size = 10 #hp_best_params['mask_size']
target_size = 20 #hp_best_params['target_size']
data_size = max_size
lr = 1e-4 #hp_best_params['lr']
batch_size = 32 #hp_best_params['batch_size']

hyper_results = test_model(network_hp_fn(epochs, masks_no, mask_size, target_size, lr=lr, batch_size=batch_size),
                    (X, y),
                    data_size,
                    None, 10)

iter 1 of 10
torch.Size([1, 128])
iter 2 of 10
torch.Size([1, 128])
iter 3 of 10
torch.Size([1, 128])
iter 4 of 10
torch.Size([1, 128])
iter 5 of 10
torch.Size([1, 128])
iter 6 of 10
torch.Size([1, 128])
iter 7 of 10
torch.Size([1, 128])
iter 8 of 10
torch.Size([1, 128])
iter 9 of 10
torch.Size([1, 128])
iter 10 of 10
torch.Size([1, 128])


In [38]:
res = hyper_results[hyper_results["Class"]=="balanced_accuracy"].reset_index(drop=True)["Metric"]
print(f"{data_size}: {res.mean():.2f} ~ {res.std():.2f}, (max: {res.max():.2f})")

99: 80.06 ~ 6.43, (max: 88.44)


### Random forest

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
def get_rf(**params):
    random_seed = np.random.randint(1024)
    def _inner():
        return RandomForestClassifier(
            random_state=random_seed,
            **params
        )
    return _inner

#### Find hyperparams

In [65]:
param_grid = {
                'n_estimators': pyhopper.int(50, 3000, multiple_of=50),
                'max_features': pyhopper.choice([None, 'sqrt', 0.2, 0.3, 0.5, 0.7]),
                'criterion' : pyhopper.choice(['gini', 'entropy']),
                'max_depth': pyhopper.choice([None, 2, 4, 8, 16]),
             }

rf_best = pyhopper_best_params(get_rf, param_grid, time="90m")
rf_best

2022-11-16 14:40:31.165 | INFO     | __main__:pyhopper_best_params:5 - pyhopper_best_params get_rf


  0%|          | [00:00<?]

Search is scheduled for 01:30:00 (h:m:s)
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5

iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter

iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter 5 of 5
iter 1 of 5
iter 2 of 5
iter 3 of 5
iter 4 of 5
iter

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Use best params

In [67]:
size = max_size

rf_dframe = test_model(get_rf(**rf_best), 
                        (X, y),
                        size,
                        None, iters=20)

iter 1 of 20
iter 2 of 20
iter 3 of 20
iter 4 of 20
iter 5 of 20
iter 6 of 20
iter 7 of 20
iter 8 of 20
iter 9 of 20
iter 10 of 20
iter 11 of 20
iter 12 of 20
iter 13 of 20
iter 14 of 20
iter 15 of 20
iter 16 of 20
iter 17 of 20
iter 18 of 20
iter 19 of 20
iter 20 of 20


# Collect analytics

In [68]:
d = {}

In [69]:
d['Random forest'] = rf_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Hypernet'] = hyper_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['HypernetPCA'] = nn_pca_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_1'] = nn1_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_2'] = nn2_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Dropout_3'] = nn3_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['Node'] = node_results.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])
d['XGBoost'] = xgb_dframe.groupby("Class")['Metric'].agg(['mean', 'std', 'max'])

In [70]:
all_models_df=pd.concat(d, axis=0)
all_models_df

mean       std         max
              Class                                             
Random forest 0                  93.000000  4.656631  100.000000
              1                  81.388889  9.061733  100.000000
              F1 score           85.012016  5.278347   94.117647
              Precision          89.779521  5.943199  100.000000
              Recall             81.388889  9.061733  100.000000
...                                    ...       ...         ...
XGBoost       Precision          88.282499  6.507296  100.000000
              Recall             80.277778  9.618283   88.888889
              Total              86.860465  2.418581   90.697674
              balanced_accuracy  85.938889  3.143415   89.666667
              roc_auc            92.933333  3.273617   98.222222

[64 rows x 3 columns]

In [71]:
all_models_df.to_csv(f"{DATA}_metrics.csv")

In [72]:
os.environ['COMET_KEY']

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [73]:
exp = Experiment(os.environ.get("COMET_KEY"), 'hypernet-uci-tune')
# exp.log_parameters({"epochs": epochs, "mask_size": mask_size, "masks_no": masks_no, "data_size": data_size})
exp.add_tag(f"hypernet-tune2{DATA}")
exp.log_table(f"{DATA}_metrics.csv", all_models_df)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/hypernet-uci-tune/14204fa2c1e94c4aa451a5adce4887b9



{'web': 'https://www.comet.com/api/asset/download?assetId=ac83e7c3ca6f42cd9382a365eae10c6e&experimentKey=14204fa2c1e94c4aa451a5adce4887b9',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=ac83e7c3ca6f42cd9382a365eae10c6e&experimentKey=14204fa2c1e94c4aa451a5adce4887b9',
 'assetId': 'ac83e7c3ca6f42cd9382a365eae10c6e'}

In [74]:
f"{DATA}_metrics.csv"

'Lymphography_metrics.csv'

In [75]:
# all_models_df = pd.read_csv(f"{DATA}_metrics.csv")

In [76]:
all_models_df

mean       std         max
              Class                                             
Random forest 0                  93.000000  4.656631  100.000000
              1                  81.388889  9.061733  100.000000
              F1 score           85.012016  5.278347   94.117647
              Precision          89.779521  5.943199  100.000000
              Recall             81.388889  9.061733  100.000000
...                                    ...       ...         ...
XGBoost       Precision          88.282499  6.507296  100.000000
              Recall             80.277778  9.618283   88.888889
              Total              86.860465  2.418581   90.697674
              balanced_accuracy  85.938889  3.143415   89.666667
              roc_auc            92.933333  3.273617   98.222222

[64 rows x 3 columns]

In [77]:
tmp = all_models_df.reset_index()

In [78]:
tmp = tmp.rename(columns={tmp.columns[0]: DATA})

In [80]:
tmp[tmp['Class'] == "balanced_accuracy"]

,Lymphography,Class,mean,std,max
6,Random forest,balanced_accuracy,87.194444,4.334289,94.444444
14,Hypernet,balanced_accuracy,83.900000,5.014974,92.444444
22,HypernetPCA,balanced_accuracy,83.177778,6.380954,100.000000
30,Dropout_1,balanced_accuracy,85.744444,5.286847,94.000000
38,Dropout_2,balanced_accuracy,84.650000,5.009189,93.222222
46,Dropout_3,balanced_accuracy,84.561111,4.414295,91.222222
54,Node,balanced_accuracy,83.933333,5.826091,92.444444
62,XGBoost,balanced_accuracy,85.938889,3.143415,89.666667
